In [1]:
import sys, os
import pandas as pd
sys.path.append('../src/')
from Biologging_Toolkit.processing.Depth import Depth

/home6/grosmaan/Documents/SES_tags/notebooks/../src/Biologging_Toolkit


In [2]:
depid = 'ml17_280a'
path = '/run/media/grosmaan/LaCie1/individus_brut/'
sens_path = os.path.join(path, 'CTD', depid, depid+'sens5.nc')
raw_path = os.path.join(path, 'individus', depid, 'raw')

In [15]:
aux = pd.read_csv(os.path.join(path, 'individus', depid, 'aux_data.csv'))  #Get existing dataframe with time array
inst = Depth(depid, 
            path = os.path.join(path, 'individus',depid), 
            raw_path = raw_path,
            sens_path = sens_path
           )
inst.dt = 5
inst.create_time(aux.time.to_numpy(), resample = True, overwrite = True)

ValueError: cannot find dimension time in this group or parent groups

In [4]:
inst.get_depth()

In [11]:
var_name = 'depth'
var_data = inst.P
time_data = inst.sens_time
overwrite = False
from scipy.interpolate import interp1d
# Overwrite the variable if it already exists
if overwrite and var_name in inst.ds.variables:
    inst.remove_variable(var_name)

# Create the dimension for time if not already present
if 'time' not in inst.ds.dimensions:
    time_data = np.arange(time_data[0], time_data[-1], self.dt)
    inst.ds.createDimension('time', len(time_data))

#Interpolation to get same timestep as reference dataframe
interp = interp1d(time_data, var_data, bounds_error = False)
var_data = interp(inst.ds['time'][:].data)

# Create or update the variable
if var_name not in inst.ds.variables:
    var = inst.ds.createVariable(var_name, np.float32, ('time',))
    var_attrs = {
        'units': 'unknown',
        'long_name': var_name
        }
    var_attrs.update(kwargs)
    for attr_name, attr_value in var_attrs.items():
        setattr(var, attr_name, attr_value)
    var[:] = var_data

In [14]:
inst.ds['time'][:].data

array([9.96920997e+36, 9.96920997e+36, 9.96920997e+36, ...,
       9.96920997e+36, 9.96920997e+36, 9.96920997e+36])

In [8]:
inst.ds['depth'][:]

masked_array(data=[nan, nan, nan, ..., nan, nan, nan],
             mask=False,
       fill_value=1e+20,
            dtype=float32)